In [1]:
import os
import json
from glob import glob
import requests
import pandas as pd
import cv2
from pdf2image import convert_from_path
from pytesseract import image_to_string
from datetime import datetime

In [2]:
pd.set_option('display.max_rows', 500)

In [3]:
import os
os.chdir(r'D:\Revinav\post-process\revinav-ocr')

In [4]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from glob import glob
import joblib
import pandas as pd

import re
import warnings
import sys
import json
from glob import glob
from datetime import datetime
from decimal import Decimal, ROUND_HALF_UP
from collections import Counter, OrderedDict
from revinavocr.core import TokenizerRegex
from revinavocr.core import common_functions
import joblib
warnings.filterwarnings("ignore")

In [269]:
from revinavocr.core.TokenizerRegex import *
from lime.lime_text import LimeTextExplainer

In [271]:
def generate_ngrams(text, n_arg):
    tokens = re.split("\\s+", text)
    ngrams = []
    for idx in range(len(tokens) - n_arg + 1):
        temp = [tokens[j] for j in range(idx, idx + n_arg)]
        ngrams.append(" ".join(temp))
    return ngrams


def remove_noise(key_word, model):
    new_key_words = {key: [] for key in key_word.keys()}

    for key, value in key_word.items():
        for item in value:
            if model.predict([item])[0] == key:
                new_key_words[key].append(item)

    return new_key_words


def get_lime(
        line, model, num_features=20, top_labels=5, ngram_args=(2, 3)
):
    lime_output = {}

    _exp = text_explainer.explain_instance(
        line, model.predict_proba, num_features=num_features, top_labels=top_labels
    )
    split_words = _exp.domain_mapper.indexed_string.inverse_vocab

    for key, value in _exp.as_map().items():
        lime_output[_exp.class_names[key]] = list(split_words[chrc[0]] for chrc in value if chrc[1] >= 0.01)

    if any(True for key, value in lime_output.items() if value):
        for _gram in ngram_args:
            for word in generate_ngrams(line, _gram):
                split_word = word.split(' ')
                for key, value in lime_output.items():
                    if all([sw in value for sw in split_word]) and model.predict([word]) == key:
                        lime_output[key].append(word)

        for key, value in lime_output.items():
            removed_count = 0
            for i in range(len(value)):
                i = i - removed_count
                for item in value[i + 1:]:
                    if value[i] in item:
                        removed_count += 1
                        lime_output[key].remove(value[i])
                        break

    lime_output = remove_noise(lime_output, model)

    return {key: value for key, value in lime_output.items() if value}

In [5]:
os.listdir(r'revinavocr/core')

['common_functions.py',
 'file_handling.py',
 'invoice_data_extraction.py',
 'load_model.py',
 'ocr2.py',
 'post_process.py',
 'regex_parser.py',
 'TokenizerRegex.py',
 '__init__.py',
 '__pycache__']

In [6]:

def create_request(
        url, files, headers=None, payload=None
):
    if not headers:
        headers = {}
    if not payload:
        payload = {}

    response = requests.request("POST", url, headers=headers, data=payload, files=files)

    return response.content


def get_content_type(ext):
    # ext = ext.remove_prefix('.')
    ext = ext.replace('.', '')

    content_type = ''
    if ext in ('jpg', 'jpeg', 'JPG', 'JPEG'):
        content_type = 'image/jpeg'

    elif ext in ('png', 'PNG'):
        content_type = 'image/png'

    elif ext in ('tsv', 'TSV'):
        content_type = 'application/octet-stream'

    return content_type


def is_pdf(doc):
    return os.path.basename(doc).split('.')[-1] in ['pdf', 'PDF']


def str2datetime(date):
    if len(date.split('/')[-1]) == 2:
        return datetime.strptime(date, '%d/%m/%y').date()
    elif len(date.split('/')[-1]) == 4:
        return datetime.strptime(date, '%d/%m/%Y').date()
    else:
        return None


def datetime2str(date):
    return '{}/{}/{}'.format(date.day, date.month, date.year)

In [7]:
img_extension='.png'
box_extension='.tsv'

In [113]:
invoice_dir = r'D:\Revinav\data\Bentex_nodata\Bentax_No_data\REV1671107437164GX'

In [114]:
invoice = glob(invoice_dir + '/*.pdf')[0]
saved_img = []

if os.path.exists(invoice):
    images = convert_from_path(invoice)

In [115]:
images

[<PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1654x2339>]

In [116]:
if images:
    doc_extension = '.' + os.path.basename(invoice).split('.')[-1]
    invoice_name = os.path.basename(invoice).replace(doc_extension, '')
    for idx, img in enumerate(images[:2]):
        img_path = os.path.join(invoice_dir, '%s-%i%s' % (invoice_name, idx, img_extension))
        img.save(img_path, 'JPEG')
        saved_img.append(img_path)

In [117]:
saved_img

['D:\\Revinav\\data\\Bentex_nodata\\Bentax_No_data\\REV1671107437164GX\\REV1671107437164GX-0.png']

In [118]:
retina_url = 'http://10.20.0.8:3030/detect'

In [119]:
retina_input = [
    ('files',
     (str(os.path.basename(img)), open(os.path.abspath(img), 'rb'), get_content_type(img_extension)))
    for img in saved_img
]

retina_output = create_request(retina_url, retina_input)
bboxes = json.loads(retina_output)


In [120]:
img = list(bboxes.keys())[0]
img

'REV1671107437164GX-0.png'

In [287]:
# for img in bboxes.keys():
box_name = os.path.join(invoice_dir, img.replace(img_extension, box_extension))
df = pd.DataFrame(json.loads(bboxes[img]))
img_path = os.path.join(invoice_dir, img)

if os.path.exists(img_path):
#     _, img_bw = cv2.threshold(cv2.imread(img_path, 0), 127, 255,
#                               cv2.THRESH_BINARY)
    img_bw = cv2.imread(img_path, 0)

    height, width = img_bw.shape
    for min_coord in ['xmin', 'ymin']:
        df[min_coord] = df[min_coord].apply(lambda x: max(1, x))

    df['xmax'] = df['xmax'].apply(lambda x: min(width, x))
    df['ymax'] = df['ymax'].apply(lambda x: min(height, x))

    df['transcript'] = df[['xmin', 'ymin', 'xmax', 'ymax']].apply(lambda x: image_to_string(
        img_bw[x[1]:x[3], x[0]:x[2]]).replace(
        '\n', ''), axis=1)
#     df = df[~df['transcript'].isnull()]

In [288]:
df = df[df['transcript'].apply(lambda x: True if x else False)]

In [289]:
df['row'] = -1
avg_width = df[['ymax', 'ymin']].apply(lambda x: abs(x[0] - x[1]), axis=1).mean()
print(avg_width)
temp_df = df.copy()

loop_count = 1
while not temp_df.empty:
    init_y = temp_df['ymin'].min()
    new_row = temp_df[temp_df['ymin'].apply(lambda x: x <= init_y + 0.3 * avg_width)].index
    df.loc[new_row, 'row'] = loop_count
    
    temp_df = df[df['row']==-1]
    loop_count += 1
    
    
df.sort_values(['row', 'xmin'], inplace=True)
df.reset_index(inplace=True, drop=True)

46.43103448275862


In [294]:
os.listdir('config')

['config.ini',
 'CreditMemo.pkl',
 'danish.pkl',
 'Date-Model.pkl',
 'dict.tsv',
 'dutch.pkl',
 'english.pkl',
 'french.pkl',
 'german.pkl',
 'response.json',
 'Trial4.pkl']

In [368]:
model = joblib.load('config/danish.pkl')
class_names = model.classes_
text_explainer = LimeTextExplainer(
    class_names=class_names, random_state=10
)
class_names

array(['ALL_AMOUNT DETAILS', 'Invoice_number', 'LINE_ITEM_KEY',
       'Payment_Term', 'Purchased_Order', 'bank_account_number',
       'bank_register_number', 'fik', 'iban', 'invoice_amount_gross',
       'invoice_amount_net', 'invoice_amount_vat', 'invoice_date',
       'invoice_due_date', 'swift_code', 'vendor_number'], dtype='<U20')

In [297]:
predict_df = pd.DataFrame(model.predict_proba(df['transcript'])*100, index = df.index, columns = model.classes_)

In [298]:
predict_df = predict_df.drop_duplicates(model.classes_, keep=False)
predict_df

,ALL_AMOUNT DETAILS,Invoice_number,LINE_ITEM_KEY,Payment_Term,Purchased_Order,bank_account_number,bank_register_number,fik,iban,invoice_amount_gross,invoice_amount_net,invoice_amount_vat,invoice_date,invoice_due_date,swift_code,vendor_number
1,0.002068,98.439090,0.008283,0.004467,0.009522,0.425424,0.614330,0.013826,0.009469,0.179520,0.132865,0.045284,0.021908,0.054385,0.033101,0.006459
3,0.000117,98.222244,0.008222,0.004454,0.009498,0.424461,0.665185,0.013791,0.009448,0.163866,0.265905,0.045175,0.021394,0.057437,0.033018,0.055784
6,0.000117,0.000044,0.008293,0.004462,0.009439,0.422922,0.663614,0.000018,0.009387,0.085184,0.126165,0.044605,98.479800,0.058639,0.032830,0.054482
12,0.000116,0.001152,0.007695,0.002886,0.006037,0.228865,0.588486,0.004316,0.008578,0.088794,0.082688,0.000498,0.012580,0.028840,0.021232,98.917237
18,0.002245,10.746588,0.183697,0.212637,0.349404,12.184116,16.503901,0.716307,0.263376,10.917552,20.269118,1.249764,1.103733,4.603642,1.646887,19.047032
26,0.000117,98.687546,0.008279,0.004483,0.009556,0.433263,0.184328,0.013883,0.009501,0.165003,0.267807,0.045422,0.021536,0.057871,0.033237,0.058167
27,0.000079,0.010553,98.034446,0.009291,0.016068,0.570568,0.844483,0.029889,0.013295,0.003260,0.038634,0.011586,0.020780,0.228176,0.069935,0.098957
28,0.000020,0.002729,98.612608,0.005294,0.011220,0.449674,0.677270,0.016909,0.010103,0.013486,0.015417,0.018853,0.005513,0.094089,0.039893,0.026923
29,0.008753,0.019329,90.446731,0.005574,0.010623,0.528470,0.818423,0.017172,0.011749,0.018076,7.950178,0.006262,0.011777,0.069623,0.041267,0.035994
30,94.185098,0.230525,0.823640,0.018000,0.016279,1.296846,1.938816,0.065569,0.029977,0.309283,0.111041,0.026783,0.120267,0.379738,0.152980,0.295159


In [346]:
# Experiment

# predict_df['primary_key'] = predict_df.index

# predict_df['max_value'] = predict_df[model.classes_].apply(lambda x: max(x), axis=1)
# predict_df['second_max']  = predict_df[model.classes_].apply(lambda x: sorted(list(x))[-2], axis=1)

# predict_df['primary_class'] = predict_df[['primary_key', 'max_value']].apply(lambda x: [cls for cls, value in \
#                                             predict_df.loc[x[0], model.classes_].items() if value==x[1]][0], axis=1)

# predict_df['secondary_class'] = predict_df[['primary_key', 'second_max']].apply(lambda x: [cls for cls, value in \
#                                             predict_df.loc[x[0], model.classes_].items() if value==x[1]][0], axis=1)


# df['foreign_key'] = df.index 

# df['primary_class'] = df['foreign_key'].apply(lambda x: predict_df.loc[x, 'primary_class'] if x in predict_df.index else None)
# df['secondary_class'] = df['foreign_key'].apply(lambda x: predict_df.loc[x, 'secondary_class'] if x in predict_df.index else None)


# result = {key: [] for key in model.classes_}
# for idx, item in df.iterrows():
#     if item['primary_class'] in extraction_dict.keys():
#         for cls in item[['primary_class', 'secondary_class']].values:
#             key = ''
#             function = extraction_dict[cls]
#             text_segment = item['transcript']

#             if cls == 'bank_register_number':
#                 value = function(text_segment, country_code)
#             else:
#                 value = function(text_segment)
                
#             if not value:
#                 y_axis = item['row']
#                 x_axis = item['xmin'] 
#                 row_df = df[(df['row']==y_axis) & (df['xmin']>x_axis)]
                
#                 if not row_df.empty:
#                     next_segment = row_df.iloc[0]['transcript']
                    
#                     if cls == 'bank_register_number':
#                         value = function(next_segment, country_code)
#                     else:
#                         value = function(next_segment)
                        
#                 if value:
#                     key = next_segment
                    
#             else:
#                 key = text_segment

#             result[cls].append((key, value))
            


In [299]:
valid_transcript_index = predict_df.index

In [320]:
df['lime_flag'] = 0
df.loc[valid_transcript_index, 'lime_flag'] = 1
df['lime_flag'] = df[['transcript', 'lime_flag']].apply(lambda x: 2 if len(str(x[0]).split(' '))>=2 
                                                           and x[1] else x[1], axis=1)

In [323]:
df

,label,xmin,ymin,xmax,ymax,transcript,row,lime_flag,predict_flag
0,0,146,274,430,321,Faktureringsadresse,1,0,False
1,0,992,272,1170,322,Faktura,1,1,True
2,0,142,315,531,362,Bodenhoffs Bagerier Herlev,2,0,False
3,0,993,318,1176,363,Fakturanr.,2,1,True
4,0,1450,315,1600,363,297388,2,0,False
5,0,148,356,402,403,Mads Mortensen,3,0,False
6,0,987,357,1127,402,Dato,3,1,True
7,0,1319,356,1603,404,14. december 2022,3,0,False
8,0,144,396,373,443,Herlev Torv 27,4,0,False
9,0,995,400,1154,444,Kundenr.,4,0,False


In [324]:
extraction_dict = {
    'Invoice_number': Inv_Number, 
    'Purchased_Order': Pur_Order,
    'bank_account_number': Bank_Acc,
    'bank_register_number': extract_bank_registration_number,
    'fik': FIK, 
    'iban': extract_iban_value, 
    'invoice_amount_gross': Amount_1,
    'invoice_amount_net': Amount_1,
    'invoice_amount_vat': Amount_1,
    'invoice_date': Date,
    'invoice_due_date': Date, 
    'swift_code': extract_swift, 
    'vendor_number': CVR

}

In [359]:
df['keywords'] = df[['transcript', 'lime_flag']].apply(lambda x: {model.predict(x[:1])[0]: [str(x[0])]} if x[1]==1 else None, axis=1)

In [369]:
df['transcript'] = df[['transcript', 'lime_flag']].apply(lambda x: re.sub('[ .,:;\|=+_\-]+', ' ', x[0]) \
                                                         if x[1]==2 else x[0], axis=1)

In [371]:
df['keywords'] = df[['transcript', 'lime_flag', 'keywords']].apply(lambda x: \
                  get_lime(x[0], model, num_features=20, top_labels=5, ngram_args=(2, 3)) \
                  if x[1]==2 else x[2], axis=1)

In [372]:
df

,label,xmin,ymin,xmax,ymax,transcript,row,lime_flag,predict_flag,keywords
0,0,146,274,430,321,Faktureringsadresse,1,0,False,None
1,0,992,272,1170,322,Faktura,1,1,True,{'Invoice_number': ['Faktura']}
2,0,142,315,531,362,Bodenhoffs Bagerier Herlev,2,0,False,None
3,0,993,318,1176,363,Fakturanr.,2,1,True,{'Invoice_number': ['Fakturanr.']}
4,0,1450,315,1600,363,297388,2,0,False,None
5,0,148,356,402,403,Mads Mortensen,3,0,False,None
6,0,987,357,1127,402,Dato,3,1,True,{'invoice_date': ['Dato']}
7,0,1319,356,1603,404,14. december 2022,3,0,False,None
8,0,144,396,373,443,Herlev Torv 27,4,0,False,None
9,0,995,400,1154,444,Kundenr.,4,0,False,None


In [373]:
df['keywords']

0                                                  None
1                       {'Invoice_number': ['Faktura']}
2                                                  None
3                    {'Invoice_number': ['Fakturanr.']}
4                                                  None
5                                                  None
6                            {'invoice_date': ['Dato']}
7                                                  None
8                                                  None
9                                                  None
10                                                 None
11                                                 None
12                    {'vendor_number': ['SE/CVR-nr.']}
13                                                 None
14                                                 None
15                                                 None
16                                                 None
17                                              

In [374]:
country_code = 'dk'

In [375]:
# result = {key: [] for key in model.classes_}
for idx, item in df.iterrows():
    keyword_dict = item['keywords']
    if keyword_dict:
        text_segment = item['transcript']

        for cls, keywords in keyword_dict.items():
            print ([(keyword in text_segment, keyword, text_segment) for keyword in keywords])


[(True, 'Faktura', 'Faktura')]
[(True, 'Fakturanr.', 'Fakturanr.')]
[(True, 'Dato', 'Dato')]
[(True, 'SE/CVR-nr.', 'SE/CVR-nr.')]
[(True, 'Vores', 'Vores ref ')]
[(True, 'Nummer', 'Nummer')]
[(True, 'Beskrivelse', 'Beskrivelse')]
[(True, 'Antal Enhed', 'Antal Enhed')]
[(True, 'Pris', 'Pris')]
[(True, 'Rabat', 'Rabat')]
[(True, 'Belob', 'Belob')]
[(True, 'for', 'Geelder for perioden 01 01 23 til')]
[(True, 'og', 'Udlans og serviceaftale')]
[(True, 'Stk', 'Stk')]
[(True, 'l alt KR', 'l alt KR ekskl moms')]
[(True, 'ekskl moms', 'l alt KR ekskl moms')]
[(True, '25', '25% moms')]
[(True, 'moms', '25% moms')]
[(True, 'alt KR inkl', 'alt KR inkl moms')]
[(True, 'Betalingsbetingelser', 'Betalingsbetingelser')]
[(True, 'Netto 8', 'Netto 8 dage')]
[(True, '8 dage', 'Netto 8 dage')]
[(True, 'CVR', 'somhedsadresse Bentax A/S Svenstrup Bane Alle 9230 Svenstrup J 70254433 CVR 27665810')]
[(True, 'Kontonr 744', 'Bank Nordjyske Bank Kontonr 744')]
[(True, 'Bank', 'Bank Nordjyske Bank Kontonr 744')]
[

In [ ]:

result = {key: [] for key in model.classes_}
for idx, item in df.iterrows():
    if item['primary_class'] in extraction_dict.keys():
        for cls in item[['primary_class', 'secondary_class']].values:
            key = ''
            function = extraction_dict[cls]
            text_segment = item['transcript']

            if cls == 'bank_register_number':
                value = function(text_segment, country_code)
            else:
                value = function(text_segment)

            if not value:
                y_axis = item['row']
                x_axis = item['xmin'] 
                row_df = df[(df['row']==y_axis) & (df['xmin']>x_axis)]

                if not row_df.empty:
                    next_segment = row_df.iloc[0]['transcript']

                    if cls == 'bank_register_number':
                        value = function(next_segment, country_code)
                    else:
                        value = function(next_segment)

                if value:
                    key = next_segment

            else:
                key = text_segment

            result[cls].append((key, value))


In [267]:
result

{'ALL_AMOUNT DETAILS': [],
 'Invoice_number': [('', []), ('297388', ['297388']), ('', [])],
 'LINE_ITEM_KEY': [],
 'Payment_Term': [],
 'Purchased_Order': [],
 'bank_account_number': [('', ''),
  ('', ''),
  ('', ''),
  ('Bank: Nordjyske Bank - Kontonr:: 7446 121 1372 - IBAN: DK8474460001:',
   [' 7446121137', '8474460001']),
  ('Bank: Nordjyske Bank - Kontonr:: 7446 121 1372 - IBAN: DK8474460001:',
   [' 7446121137', '8474460001'])],
 'bank_register_number': [('', ''),
  ('', ''),
  ('14. december 2022', ['2022']),
  ('', ''),
  ('', ''),
  ('', ''),
  ('', ''),
  ('', ''),
  ('somhedsadresse Bentax A/S Svenstrup Bane Alle 9230 Svenstrup J - 70254433 - CVR: 27665810',
   ['9230']),
  ('Bank: Nordjyske Bank - Kontonr:: 7446 121 1372 - IBAN: DK8474460001:',
   ['7446', '1372']),
  ('Bank: Nordjyske Bank - Kontonr:: 7446 121 1372 - IBAN: DK8474460001:',
   ['7446', '1372'])],
 'fik': [('', '')],
 'iban': [('2 - IBAN: DK8474460001211372 - SWIFT-kode: RINGDK22',
   ['DK8474460001211372'])]

In [ ]:
def lime_output(txt):
    

In [241]:
## For Vaishnavi

key_df = value_df = pd.DataFrame(index=range(len(lines)), columns= ['line'] + list(class_names))

for idx, line in enumerate(lines):
    line = re.sub('[ .,:;\|=+_\-]+', ' ', line)
    key_df.loc[idx, 'line'] = line
#     value_df.loc[idx, 'line'] = line
    lime_output = get_lime(
        line, model, num_features=20, top_labels=5, ngram_args=(2, 3)
)
    for cls, words in lime_output.items():
        if words and cls in key_df.columns:
            key_df.loc[idx, cls] = tuple(words)
    

array(['Invoice_number', 'bank_register_number'], dtype=object)

In [336]:
text_segment = '2 - IBAN: DK8474460001211372 - SWIFT-kode: RINGDK22'

In [337]:
get_lime(
        re.sub('[ .,:;\|=+_\-]+', ' ', x[0]), model, num_features=20, top_labels=5, ngram_args=(2, 3)
)

{'fik': ['kode'], 'iban': ['IBAN'], 'swift_code': ['SWIFT']}